In [1]:
import h3
import geopandas as gpd
import pandas as pd


# Define a function to convert lat/lon to H3 index at a specific resolution
# def point_to_h3(point, resolution=8):
#     return h3.geo_to_h3(point.y, point.x, resolution)

In [2]:
cities_path = (
    "/Users/test/Documents/code/IL2/factory_loc_service/api/app/data/cities.parquet"
)
cities = gpd.read_parquet(cities_path)

In [3]:
cities.head(1)

,region_city,city_category,population,harsh_climate,ueqi_score,ueqi_residential,ueqi_street_networks,ueqi_green_spaces,ueqi_public_and_business_infrastructure,ueqi_social_and_leisure_infrastructure,...,probability_to_move,probability_to_move_to_SELECTED_CITY,migrations_to_selected_city,num_in_migration,one_vacancy_out_response,estimate,h3_index,x,y,geometry
0,"Хакасия, Абаза",Малый город,12055,False,172,36,34,27,16,20,...,0.889793,0.002423,0.029192,0.39094,74.18,0.512,862535cc7ffffff,90.097573,52.647106,POINT (90.09757 52.64711)


In [4]:
"""
(уже пред-посчитано)
*Тоже считается только один раз
"""

# Convert cities to H3 indices

# resolution = 6  # Choose a resolution based on your needs
# cities["h3_index"] = cities.geometry.apply(lambda x: point_to_h3(x, resolution))

'\n(уже пред-посчитано)\n*Тоже считается только один раз\n'

In [5]:
import pandas as pd
import folium
from shapely.geometry import Point


def get_n_closest_cities(
    gdf: pd.DataFrame, target_hex_idx: str, n_hex_radius: int = 20
):
    ZERO_DF_LEN = 0
    # Find nearby hexagons within a K-ring (radius in hexes)
    nearby_hexes_list = h3.k_ring(target_hex_idx, k=n_hex_radius)

    assert len(nearby_hexes_list) > ZERO_DF_LEN, "No cities in provided radius"

    nearby_hexes_mask = gdf["h3_index"].isin(nearby_hexes_list)

    assert sum(nearby_hexes_mask) >= 1, f"No cities with that h3 idx {target_hex_idx}"

    return gdf.loc[nearby_hexes_mask, :]


def get_city_h3_idx_by_name(df: pd.DataFrame, target_city_name: str) -> str:

    MIN_DF_LEN = 1
    ZERO_DF_LEN = 0

    assert len(df) > ZERO_DF_LEN, "Empty df"
    assert len(target_city_name) > ZERO_DF_LEN, "No city name provided"

    mask_city_name = df["region_city"] == target_city_name

    assert sum(mask_city_name) >= MIN_DF_LEN, "No city in a table"

    h3_hex_idx = df.loc[mask_city_name, "h3_index"].item()

    return h3_hex_idx


# Define a function to convert lat/lon to H3 index at a specific resolution
def point_to_h3(point, resolution):
    return h3.geo_to_h3(point.y, point.x, resolution)


# Function to find N closest cities using H3 k-ring
def find_n_closest_cities(point, gdf_cities, hours, resolution, speed_km_per_hour=80):

    FIX_COEFF = 2.5
    # Convert hours to kilometers
    h_to_km = hours * speed_km_per_hour

    # Calculate the average distance of a hexagon at the given resolution
    hex_size_km = 4 * FIX_COEFF  # Get the radius of the hexagon in kilometers
    n_hex = int(
        h_to_km / hex_size_km
    )  # Calculate the number of hexagons that fit in the distance

    # Convert the point to an H3 index
    point_h3 = point_to_h3(point, resolution)
    # print(d)

    nearby_cities = get_n_closest_cities(gdf_cities, point_h3, n_hex)

    # Return the N closest cities (no distance calculation, just based on H3 proximity)
    return nearby_cities


def explore_closest_cities(point, closest_cities):
    # Find the N closest cities

    # Create a GeoDataFrame for the target point
    # gdf_point = gpd.GeoDataFrame(geometry=[point], crs=gdf_cities.crs)

    # Create a base map centered on the point
    m = folium.Map(location=[point.y, point.x], zoom_start=10)

    # Add the target point to the map
    folium.Marker(
        [point.y, point.x], popup="Target Point", icon=folium.Icon(color="red")
    ).add_to(m)

    # Add the closest cities to the map
    folium.GeoJson(closest_cities.to_crs(3857).buffer(5000).to_crs(4326)).add_to(m)

    # You can add more layers or styles here if needed
    return m

In [18]:
# Example usage with a test point (Moscow coordinates)
test_point = Point(40.627537, 47.589869)  # (lon, lat),
resolution = 6
n_hours = 1.5

%time
# Convert all city points to H3 indices
cities["h3_index"] = cities.geometry.apply(lambda x: point_to_h3(x, resolution))
closest_cities = find_n_closest_cities(test_point, cities, n_hours, resolution)
display(closest_cities)

CPU times: user 2 μs, sys: 1 μs, total: 3 μs
Wall time: 5.96 μs


,region_city,city_category,population,harsh_climate,ueqi_score,ueqi_residential,ueqi_street_networks,ueqi_green_spaces,ueqi_public_and_business_infrastructure,ueqi_social_and_leisure_infrastructure,...,probability_to_move,probability_to_move_to_SELECTED_CITY,migrations_to_selected_city,num_in_migration,one_vacancy_out_response,estimate,h3_index,x,y,geometry
9,"Ростовская область, Аксай",Малый город,48714,False,212,51,31,26,35,26,...,-0.233261,0.022052,0.108949,39.813440,2.210,0.517,862d44397ffffff,39.86,47.27,POINT (39.86000 47.27000)
70,"Ростовская область, Белая Калитва",Малый город,39457,False,193,37,32,31,23,36,...,0.473569,0.004197,0.029191,3.234763,34.933,0.523,862d453afffffff,40.80,48.18,POINT (40.80000 48.18000)
287,"Ростовская область, Зверево",Малый город,18718,False,177,37,37,22,14,29,...,0.019607,0.006773,0.029191,9.721367,5.966,0.440,862d45137ffffff,40.13,48.04,POINT (40.13000 48.04000)
396,"Ростовская область, Константиновск",Малый город,16910,False,178,35,35,23,24,18,...,0.802214,0.003345,0.029192,0.968655,16.518,0.429,862d40a6fffffff,41.10,47.58,POINT (41.10000 47.58000)
429,"Ростовская область, Красный Сулин",Малый город,35347,False,168,41,28,14,12,39,...,0.248196,0.004614,0.029191,5.078478,26.977,0.514,862d45c57ffffff,40.06,47.89,POINT (40.06000 47.89000)
613,"Ростовская область, Новочеркасск",Большой город,160782,False,195,39,31,31,25,28,...,0.326694,0.106652,1.223430,105.125577,5.907,0.814,862d442e7ffffff,40.11,47.41,POINT (40.11000 47.41000)
614,"Ростовская область, Новошахтинск",Большой город,101708,False,139,28,25,18,13,23,...,0.246491,0.029674,0.829630,32.733001,1.589,0.530,862d45c17ffffff,39.93,47.77,POINT (39.93000 47.77000)
770,"Ростовская область, Семикаракорск",Малый город,21488,False,180,40,39,24,16,25,...,0.806558,0.003617,0.029191,1.024335,75.171,0.567,862d40b77ffffff,40.81,47.52,POINT (40.81000 47.52000)
963,"Ростовская область, Шахты",Большой город,222489,False,161,23,26,20,22,30,...,0.278661,0.063002,1.223430,66.530985,5.817,0.741,862d4552fffffff,40.22,47.71,POINT (40.22000 47.71000)


In [19]:
closest_cities.to_feather("closest_cities.feather")